


# DistFlow implementation

Based on the implementation in source [2], with a little help from source [3] (see below)

## Overview
The DistFlow formulation consist of several equations that describe the flow of power through the network. The following notation is used:
- $y_m = |V_m|^2$: voltage at node $m$ squared
- $u_{n}^{p,q}$: The free power (real ($p$) and reactive ($q$)) at node $n$. Free power means the DSO has the ability to change this
- $p_{n}$: The uncontrollable real power at node $n$. 
- $q_{n}$: The uncontrollable reactive power at node $n$. 
- $l_{mn} = |I_{mn}|^2$: Current from $m$ to $n$ squared
- $P_{mn}$: Real power from $m$ to $n$
- $Q_{mn}$: Reactive power from $m$ to $n$
- $Q_{mn}$: Reactive power from $m$ to $n$
- $r_{mn}$: Resistance over line from node $m$ to $n$
- $x_{mn}$: Reactance over line from node $m$ to $n$
- $E$: The set of all edges in the topology; all connecting graphs 
  

### Power in = Power out
Equation (1) from source [2] describes the real power flow:
$$P_{mn} = \underbrace{r_{mn}l_{mn}}_{\text{line losses from m to n}} + \underbrace{p_n}_{\text{net uncontrollable power at node n}} + \underbrace{u_n^p}_{\text{net controllable power at node n}} + \underbrace{\sum_{(n,k)\in E, k\neq m}P_{nk}}_{\text{power going from node n to all connected nodes except m}} \tag{1}$$
I've slightly changed the original eq. (1) since they use $p_n = p_n^g + p_n^c$ to seperate production and consumption. Since those are predetermined anyway I've added them together.

Equation (2) from source [2] describes the reactive power flow:
$$Q_{mn} = \underbrace{x_{mn}l_{mn}}_{\text{line losses from m to n}} + \underbrace{q_n}_{\text{net uncontrollable power at node n}} + \underbrace{u_n^q}_{\text{net controllable power at node n}} + \underbrace{\sum_{(n,k)\in E, k\neq m}Q_{nk}}_{\text{power going from node n to all connected nodes except m}}\tag{2}$$

### Voltage relation
Equation (3) from source [2] describes the voltage relation:
$$\underbrace{y_m}_{v_m^2} = \underbrace{y_n}_{v_n^2} + \underbrace{2(r_{mn}P_{mn} + x_{mn}Q_{mn})}_{\text{not sure; voltage rise due to forced power?}} - \underbrace{(r_{mn}^2 + x_{mn}^2)l_{mn}}_{\text{current induced voltage drop over line $(m,n)$}} \tag{3}$$


### Voltage limitations
Equation (7) in source [2] describes the voltage limitation:
$$\underline{y}_m \leq y \leq \overline{y}_m \hspace{1em} \forall \hspace{1em} m \in N$$


### Current limitations
Equation (3) from source [2] describes the current limits:
$$\underbrace{l_{mn}}_{I_{mn}^2} = (\underbrace{P_{mn}^2 + Q_{mn}^2}_{\text{powers going through } (m,n)})/\underbrace{y_m}_{v_m^2}$$
To conform to a second order cone program, this is relaxed to
$$l_{mn} \geq (P_{mn}^2 + Q_{mn}^2)/y_m \tag{6}$$
which can (according to [3].IV) be rewritten to the Second order cone constraint
$$\begin{bmatrix}2P_{mn} \\  2Q_{mn} \\ l_{mn} - y_m\end{bmatrix}_2 =  l_{mn} + y_m \tag{6B}$$
where subscript $2$ indicates that you need to take the l2 norm of the vector.

(6B) is equivalant to (6) since
$$\sqrt{4P_{mn}^2 + 4Q_{mn}^2 + (l_{mn} - y_m)^2} = l_{mn} + y_m \text{   (this is equation 6B)}$$
$$4P_{mn}^2 + 4Q_{mn}^2 + (l_{mn} - y_m)^2 = (l_{mn} + y_m)^2$$
$$4P_{mn}^2 + 4Q_{mn}^2 = (l_{mn} + y_m)^2 - (l_{mn} - y_m)^2$$
$$4P_{mn}^2 + 4Q_{mn}^2 = (l_{mn}^2 + y_m^2 + 2l_{mn}y_m) - (l_{mn}^2 + y_m^2 -2l_{mn}y_m)$$
$$4P_{mn}^2 + 4Q_{mn}^2 = 4l_{mn}y_m$$
$$P_{mn}^2 + Q_{mn}^2 = l_{mn}y_m$$
$$\frac{P_{mn}^2 + Q_{mn}^2}{y_m} = l_{mn} \text{   (this is equaition 6)}$$
## Optimization formulation
### DECISION VARIABLES
- Capital P: The real power over lines
- Capital Q: The reactive power over lines
- small l: square of current over lines

- small u_p: free real power at nodes
- small u_q: free reactive power at nodes
- small y: square of voltage at nodes

### PARAMETERS
Parameters are like constants used by cvxpy.
- small p: The predetermined uncontrollable net real power consumption per node
- small q: The predetermined uncontrollable net reactive power consumption per node

### CONSTRAINTS
See equations  (1), (2), (3), (7), (6)

CVXPY requires a specific way to write (6), namely (6B)

### OBJECTIVES
Source [2] describes the following objectives:
- Power loss over lines $$\sum_{(m,n)\in E}r_{mn}l_{mn}$$
- Deviations from reference voltage $$\sum_{n\in N}(y_n-y_{ref})^2$$
- Power delivered by transmission grid $$P_{01}^2 + Q_{01}^2$$

In the paper they use a linear combination of these three objectives

## Sources
- [1] Convex Relaxations of Optimal Power Flow, Part I. By Steven Low, https://arxiv.org/abs/1405.0766
- [2] Towards Distributed Energy Services: Decentralizing Optimal Power Flow with Machine Learning, By Roel Dobbe, https://arxiv.org/pdf/1806.06790.pdf
- [3] Branch Flow Model: Relaxations and Convexification (Part I), By Masoud Farivar and Steven Low https://arxiv.org/pdf/1204.4865.pdf




In [6]:
import cvxpy as cp
import numpy as np
import utils.network_utilities as nut

In [7]:
network = nut.make_simple_mesh_net()

INCLUDE_POWER_QUALITY_CONSTRAINT = True

In [8]:
### DECISION VARIABLES
# free variables
u_r = cp.Variable(len(network["node"]["id"])) # Free real power per node
u_q = cp.Variable(len(network["node"]["id"])) # Free reactive power per node

# dependant variables; no clue how to model these otherwise
P = cp.Variable(len(network["line"]["id"])) # CAPITAL P; Real power flow over line
Q = cp.Variable(len(network["line"]["id"])) # CAPITAL Q; Reactive power flow over line
l = cp.Variable(len(network["line"]["id"]), nonneg=True) # I^2 over line, nonneg since square
y = cp.Variable(len(network["node"]["id"]), nonneg=True) # V^2 per node, nonneg since square and voltage

### PARAMETERS (i.e. constants)
p = cp.Parameter(len(network["node"]["id"]), value=network["node"]["p_specified"]) # actual real power per node.
q = cp.Parameter(len(network["node"]["id"]), value=network["node"]["q_specified"]) # actual reactive power per node

In [10]:
constraints = []

# go over all lines:
for i, line_id in enumerate(network["line"]["id"]):
    to_node_id = network["line"]["to_node"][i]
    from_node_id = network["line"]["from_node"][i]

    # determine indices corresponding to the current node ids
    to_node_idx = nut.id_to_index(network["node"], to_node_id)
    from_node_idx = nut.id_to_index(network["node"], from_node_id)

    # real power flow equality
    # Source [1], eq. (1) 
    constraints += [P[i] == network["line"]["r1"][i]*l[i] \
                    + p[to_node_idx]\
                    + u_r[to_node_idx]\
                    + cp.sum([P[j] for j in network["line"]["outgoing_line_indices"][i]]) \
                    - cp.sum([P[j] for j in network["line"]["outgoing_line_indices"][i]])] # (eq. (1))

    # reactive power flow equality
    # Source [1], eq. (2)
    constraints += [Q[i] == network["line"]["x1"][i]*l[i] \
                    + q[to_node_idx]\
                    + u_q[to_node_idx] \
                    + cp.sum([Q[j] for j in network["line"]["outgoing_line_indices"][i]]) \
                    - cp.sum([Q[j] for j in network["line"]["outgoing_line_indices"][i]])] # (eq. (3))

    
    # Line limits
    # See eq. 6b above. Corresponds to source [1] eq. 6 and source [3] sectionIV 
    # ACCORDING TO CVXPY DOC: "We use cp.SOC(t, x) to create the SOC constraint ||x||_2 <= t"
    constraints += [cp.SOC(l[i]+y[to_node_idx], cp.vstack([P[i],Q[i],l[i]-y[to_node_idx]]))]

# go over all nodes:
for i, node_id in enumerate(network["node"]["id"]):
    downstream_node_indices = network["node"]["downstream_node_indices"][i]
    downstream_node_ids = network["node"]["downstream_node_ids"][i]

    # voltage constraints eq. (7)
    constraints += [y[i] <= network["node"]["u_max"][i]**2]
    constraints += [y[i] >= network["node"]["u_min"][i]**2]

    if INCLUDE_POWER_QUALITY_CONSTRAINT:
        PF = 0.85
        f_q = PF
        f_r = np.sqrt(1-PF**2)

        constraints += [cp.SOC(0, cp.vstack([f_r*(p[i]+u_r[i]),f_q*(q[i]+u_q[i])]))]

    for downstream_node_index, downstream_node_id in zip(downstream_node_indices, downstream_node_ids):
        # get the index of the line with from_node=node_id and to_node=downstream_node_id
        line_idx = nut.node_ids_to_line_index(upstream_node=node_id, downstream_node=downstream_node_id, line=network["line"])

        # voltage constraint,
        #  eq. (3) 
        constraints += [y[i] == y[downstream_node_index] + 2*(network["line"]["r1"][line_idx]*P[line_idx] + network["line"]["x1"][line_idx]*Q[line_idx]) - (network["line"]["r1"][line_idx]**2 + network["line"]["x1"][line_idx]**2)*l[line_idx]]

        

In [11]:
objective = cp.Minimize(cp.sum(y))
problem = cp.Problem(objective, constraints)

In [12]:
problem.solve()

ParameterError: A Parameter (whose name is 'param12') does not have a value associated with it; all Parameter objects must have values before solving a problem.